## Installing Packages 
    1. Pandas
    2. Requests
    3. Pandasql

In [1]:
! pip install pandas requests pandasql

You are using pip version 19.0.3, however version 22.2.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Importing the packages

In [2]:
import requests
import pandas as pd
from pandasql import sqldf

## Creating a function to compute age:

This link https://www.hockey-reference.com/about/glossary.html explains that for each season, the age 
of the player should be calculated from the birthdate till jan 31 of the season year. 

So, here we are creating a function called get_age which takes season and birthday as the string
in the following forms:

    1. season = 20212022
    2. birthday = "1985-01-23"

So, we create two datetime objects for season (with jan 31 as the base date) and for the birthday.

We just compute the differencere between these two to get the age. 


In [3]:
from datetime import datetime

def get_age(season, birthday):
    season_end_age = season[:4] + "-01" + "-31"
    current_time = datetime.strptime(season_end_age, "%Y-%m-%d")
    bday = datetime.strptime(birthday, "%Y-%m-%d")

    diff = current_time - bday
    
    age_years = diff.days // 365
    age_days = diff.days % 365
    
    frac = age_days / 365
    final_age = age_years + frac
    return final_age

## Function.1 Create_panthers_players_table

So, here the idea is "that we will be getting the data using an open API which is provided by
the NFL. Every API has something known as endpoint (just a fancy way of saying URL). 

This api's base URL looks something like this: https://statsapi.web.nhl.com/api/v1

Now, there are various endpoints such as:

    1. https://statsapi.web.nhl.com/api/v1/teams
    2. https://statsapi.web.nhl.com/api/v1/people
    3. https://statsapi.web.nhl.com/api/v1/game
    4. https://statsapi.web.nhl.com/api/v1/schedule
    
For this function, we are just going to need **teams** and **people** endpoint.

First we created a list of seasons and then in a loop extracting the data for each season. We do the following for each season:

        1. We append the season to the base link for teams and we also filter for only
           florida panthers data by giving the floarida team id = 13. 
        2. We get the json of the of the created link using the requests package. 
        3. Then, extract the players which is stored in the roster key
        4. We then loop through each player and store the basic info and their height and weight
        from the dictionary objects
        5. In the inside loop, we also use the get_age function which we created earlier.
        6. Finally, we create a subset data frame and keep on appending it to the base one.
        7. and after the loop ends, we return the dataframe.


In [4]:
def create_panthers_players_table():

    seasons = ["20142015", "20152016", "20162017", "20172018", "20182019",
              "20192020","20202021", "20212022"]
    roster_link = "https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster&season="
    florida_team_id = 13

    panthers_players_table = pd.DataFrame()

    for season in seasons:
        print("Querying data for season = ", season)
        current_roster = requests.get(roster_link + season + "&teamId=" + str(florida_team_id))
        current_roster = current_roster.json()

        list_of_players = current_roster["teams"][0]["roster"]["roster"]

        for player in list_of_players:
            current_id = player["person"]["id"]

            player_info_link = "https://statsapi.web.nhl.com/api/v1/people/"+str(current_id)
            player_stats_link = "https://statsapi.web.nhl.com/api/v1/people/" + str(current_id) + "/stats?stats=statsSingleSeason&season="+season

            player_info = requests.get(player_info_link).json()
            player_stats = requests.get(player_stats_link).json()

            height = player_info["people"][0]["height"]
            weight = player_info["people"][0]["height"]
            birthday = player_info["people"][0]["birthDate"]

            player_stats = player_stats["stats"][0]["splits"]
            if len(player_stats) == 0:
                print(2+2)
            else:
                player_stats = player_stats[0]["stat"]
                age = get_age(season, birthday)


                keys = ["id", "season", "height", "weight", "age"] + list(player_stats.keys())
                values = [current_id, season, height, weight, age] + list(player_stats.values())

                current_data = pd.DataFrame([values], columns = keys)
                panthers_players_table = pd.concat((panthers_players_table, current_data))
    panthers_players_table.reset_index(inplace = True, drop = True)
    return panthers_players_table

## Call the function and Save the data

Here, we are calling the function we just created and saving it as csv. 

In [5]:
panthers_players_table = create_panthers_players_table()
panthers_players_table.to_csv("panthers_players_table.csv", index = False)

Querying data for season =  20142015
Querying data for season =  20152016
Querying data for season =  20162017
Querying data for season =  20172018
Querying data for season =  20182019
Querying data for season =  20192020
4
Querying data for season =  20202021
Querying data for season =  20212022


## Function.2 create_panthers_game_boxscores_table

The process is just mostly same as the previous one. We loop through the seasons and get the 
appropriate keys from the jsons. 

In [6]:
def create_panthers_game_boxscores_table():
    seasons = ["20142015", "20152016", "20162017", "20172018", "20182019",
               "20192020","20202021", "20212022"]

    df = pd.DataFrame()

    for season in seasons:
        print("Querying data for season = ", season)
        link = f"https://statsapi.web.nhl.com/api/v1/schedule?season={season}&teamId=13"
        json = requests.get(link).json()
        list_of_matches = json["dates"]
        for match in list_of_matches:
            game_id = match["games"][0]["gamePk"]
            h_or_a = match["games"][0]["teams"]["away"]["team"]["id"]
            home_or_away = "away" if h_or_a == 13 else "home"
            game_link = f"https://statsapi.web.nhl.com/api/v1/game/{game_id}/boxscore"
            game_stats = requests.get(game_link).json()

            panthers_stats = game_stats["teams"][home_or_away]["teamStats"]["teamSkaterStats"]
            oppo_h_or_a = "home" if home_or_away == "away" else "home"
            opponent_stats = game_stats["teams"][oppo_h_or_a]["teamStats"]["teamSkaterStats"]

            overall_stats_dict = {"panthers_"+key:[value] for key,value in panthers_stats.items()}

            overall_stats_dict.update({"opponent_"+key:[value] for key,value in opponent_stats.items()})


            final_Dict = {"game_id":[game_id], "HomeOrAway":[home_or_away]}
            final_Dict.update(overall_stats_dict)  

            subset_df = pd.DataFrame(final_Dict)
            df = pd.concat((df, subset_df))
    df.reset_index(inplace = True, drop = True)
    return df

## Call the function and Save the data

Here, we are calling the function we just created and saving it as csv. 

In [7]:
panthers_game_boxscores = create_panthers_game_boxscores_table()
panthers_game_boxscores.to_csv("panthers_game_boxscores.csv", index = False)

Querying data for season =  20142015
Querying data for season =  20152016
Querying data for season =  20162017
Querying data for season =  20172018
Querying data for season =  20182019
Querying data for season =  20192020
Querying data for season =  20202021
Querying data for season =  20212022


## Function.3 create_panthers_player_boxscores_table

In [8]:
def create_panthers_player_boxscores_table():
    d = pd.read_csv("panthers_game_boxscores.csv")
    
    game_ids = d.game_id.values
    home_or_away = d.HomeOrAway.values

    panthers_player_boxscores_table = pd.DataFrame()

    for i,g_id in enumerate(game_ids):
        print("Getting data for game id = ", g_id)
        current_link = f"https://statsapi.web.nhl.com/api/v1/game/{g_id}/boxscore"
        json = requests.get(current_link).json()
        players = json["teams"][home_or_away[i]]["players"].keys()

        for player_id in players:
            curr_player = json["teams"][home_or_away[i]]["players"][player_id]
            stats = curr_player["stats"]

            dict_to_update = {"game_id":[g_id], "player_id":[player_id]}
            if len(stats.keys()) == 0:
                pass
            else:
                keys_of_stats = stats.keys()
                final_stats = {}
                for k in keys_of_stats:
                    new_stats = stats[k]
                    final_stats.update({k + "_" + ke:[va] for ke,va in new_stats.items()})

                dict_to_update.update(final_stats)
                subset_df = pd.DataFrame(dict_to_update)
                panthers_player_boxscores_table = pd.concat((panthers_player_boxscores_table, subset_df))
    panthers_player_boxscores_table.reset_index(inplace=True, drop = True)
    return panthers_player_boxscores_table

## Call the function and Save the data

Here, we are calling the function we just created and saving it as csv. 

In [9]:
panthers_players_boxscores = create_panthers_player_boxscores_table()
panthers_players_boxscores.to_csv("panthers_players_boxscores.csv", index = False)

Getting data for game id =  2014010029
Getting data for game id =  2014010051
Getting data for game id =  2014010065
Getting data for game id =  2014010088
Getting data for game id =  2014010100
Getting data for game id =  2014020010
Getting data for game id =  2014020022
Getting data for game id =  2014020037
Getting data for game id =  2014020056
Getting data for game id =  2014020069
Getting data for game id =  2014020087
Getting data for game id =  2014020115
Getting data for game id =  2014020143
Getting data for game id =  2014020156
Getting data for game id =  2014020172
Getting data for game id =  2014020188
Getting data for game id =  2014020202
Getting data for game id =  2014020225
Getting data for game id =  2014020246
Getting data for game id =  2014020264
Getting data for game id =  2014020277
Getting data for game id =  2014020290
Getting data for game id =  2014020302
Getting data for game id =  2014020315
Getting data for game id =  2014020327
Getting data for game id 

Getting data for game id =  2016020385
Getting data for game id =  2016020400
Getting data for game id =  2016020416
Getting data for game id =  2016020438
Getting data for game id =  2016020451
Getting data for game id =  2016020457
Getting data for game id =  2016020487
Getting data for game id =  2016020501
Getting data for game id =  2016020510
Getting data for game id =  2016020527
Getting data for game id =  2016020536
Getting data for game id =  2016020554
Getting data for game id =  2016020572
Getting data for game id =  2016020585
Getting data for game id =  2016020596
Getting data for game id =  2016020609
Getting data for game id =  2016020620
Getting data for game id =  2016020636
Getting data for game id =  2016020645
Getting data for game id =  2016020669
Getting data for game id =  2016020675
Getting data for game id =  2016020690
Getting data for game id =  2016020713
Getting data for game id =  2016020734
Getting data for game id =  2016020748
Getting data for game id 

Getting data for game id =  2018020989
Getting data for game id =  2018021005
Getting data for game id =  2018021013
Getting data for game id =  2018021022
Getting data for game id =  2018021033
Getting data for game id =  2018021044
Getting data for game id =  2018021060
Getting data for game id =  2018021092
Getting data for game id =  2018021101
Getting data for game id =  2018021116
Getting data for game id =  2018021130
Getting data for game id =  2018021136
Getting data for game id =  2018021156
Getting data for game id =  2018021169
Getting data for game id =  2018021180
Getting data for game id =  2018021192
Getting data for game id =  2018021204
Getting data for game id =  2018021221
Getting data for game id =  2018021244
Getting data for game id =  2018021262
Getting data for game id =  2019010002
Getting data for game id =  2019010024
Getting data for game id =  2019010034
Getting data for game id =  2019010051
Getting data for game id =  2019010067
Getting data for game id 

Getting data for game id =  2021020749
Getting data for game id =  2021020760
Getting data for game id =  2021020776
Getting data for game id =  2021020793
Getting data for game id =  2021020800
Getting data for game id =  2021020527
Getting data for game id =  2021020476
Getting data for game id =  2021020498
Getting data for game id =  2021020511
Getting data for game id =  2021020816
Getting data for game id =  2021020831
Getting data for game id =  2021020865
Getting data for game id =  2021020881
Getting data for game id =  2021020899
Getting data for game id =  2021020906
Getting data for game id =  2021020920
Getting data for game id =  2021020951
Getting data for game id =  2021020963
Getting data for game id =  2021020975
Getting data for game id =  2021020981
Getting data for game id =  2021021020
Getting data for game id =  2021021036
Getting data for game id =  2021021046
Getting data for game id =  2021021061
Getting data for game id =  2021021076
Getting data for game id 

## SQL

## Load the data we just created

Now, we will run some sql queries on the data which we just created. First, let us load the data in three variables

In [10]:
panthers_players_boxscores = pd.read_csv("panthers_players_boxscores.csv")
panthers_game_boxscores = pd.read_csv("panthers_game_boxscores.csv")
panthers_players_table = pd.read_csv("panthers_players_table.csv")

In [11]:
pysqldf = lambda q: sqldf(q, globals())

## Question.1 

Here, we are querying the panthers_players_boxscores and doing a group by on season and player_id and counting all the instances for each group by. This way, we will be able to look at all those players
who had the most seasons. 

In [13]:
q1 = '''
select season,
    player_id,
    count(*) as count from (select game_id,game_id/1000000 as season,player_id
    from panthers_players_boxscores) as p
group by p.season,p.player_id
order by count desc 
'''
q1_answer = pysqldf(q1)
print(q1_answer.head(20))

    season  player_id  count
0     2021  ID8476456     94
1     2021  ID8477346     94
2     2021  ID8477986     94
3     2021  ID8480185     93
4     2021  ID8477933     92
5     2015  ID8466285     91
6     2015  ID8469638     91
7     2015  ID8475191     91
8     2021  ID8475462     91
9     2021  ID8477409     90
10    2015  ID8448208     88
11    2015  ID8477932     87
12    2018  ID8471735     86
13    2018  ID8474884     86
14    2018  ID8476456     86
15    2021  ID8477407     86
16    2014  ID8466285     85
17    2015  ID8476456     85
18    2017  ID8471735     85
19    2017  ID8475760     85


## Question.2

We are querying the panthers_players_boxscores and ordering the data by skaterStats_plusMinus 
and only getting the top row with the maximum plus minus score.

In [14]:
q2 = '''
select game_id,skaterStats_plusMinus from panthers_players_boxscores
order by skaterStats_plusMinus desc
limit 1
'''
q2_answer = pysqldf(q2)
print(q2_answer)

      game_id  skaterStats_plusMinus
0  2021020661                    6.0


## Question.3

Here, we want to get the information on the youngest player from the latest season 2021-2022. We get the season by using teh where clause and then sort the data by age column. 

In [15]:
q3 = '''
select id, age from panthers_players_table
where season = '20212022'
order by age
'''
q3_answer = pysqldf(q3)
print(q3_answer.head())

        id        age
0  8482113  19.342466
1  8481655  19.783562
2  8481519  19.800000
3  8480844  20.619178
4  8480015  21.972603


## Question.4 

This is a complicated query. Let us try to understand this one by one. 

There is a sub query first.  Here, we are getting the max of penalty minutes from the panthers_players_boxscores and then in the main query, we are getting the player_id and game_id
for which penality minutes was maximum. 


In [16]:
q4 = '''
select player_id,game_id,skaterStats_penaltyMinutes 
from panthers_players_boxscores
where skaterStats_penaltyMinutes in (select max(skaterStats_penaltyMinutes) from panthers_players_boxscores)
'''
q4_answer = pysqldf(q4)
print(q4_answer)

   player_id     game_id  skaterStats_penaltyMinutes
0  ID8474230  2017020738                        39.0


## Question.5

We are extracting the game id from table panthers_game_boxscores and only getting he "Home"games with
opponent goals = 0.

In [17]:
q5 = '''
select game_id
from panthers_game_boxscores
where HomeOrAway = 'home' and opponent_goals = 0
'''
q5_answer = pysqldf(q5)
print(q5_answer)

       game_id
0   2014010088
1   2014020037
2   2014020912
3   2014021172
4   2015020827
5   2016020596
6   2018010005
7   2018020865
8   2019021012
9   2019011004
10  2020020303
